In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/data-full.csv", index_col=0)

In [3]:
df.head(2)

,Пол,День рождения,Возраст клиента,Заявка №,Дата бронирования,Дата начала тура,Год,Ночей,Вид тура,Тип заявки,...,Младенцы,Звездность,Состав группы (Заявка) (Заявка),Сумма в $,Наименование тура,ИД клиента,Глубина продаж,Тип питания,Город Отеля,Город отправления
0,мужской,2018-08-28 00:00:00,2.0,4803671,2021-02-07 20:44:00,2021-05-04 00:00:00,2021,12,Пляжный отдых,Пакет,...,0,4*,Семья,1718.64,Turkey Antalya LED,4197428,85.136111,All Inclusive,NaN,Санкт-Петербург
1,мужской,1991-04-25 00:00:00,29.0,4803671,2021-02-07 20:44:00,2021-05-04 00:00:00,2021,12,Пляжный отдых,Пакет,...,0,4*,Семья,1718.64,Turkey Antalya LED,4197429,85.136111,All Inclusive,NaN,Санкт-Петербург


In [4]:
df.columns

Index(['Пол', 'День рождения', 'Возраст клиента', 'Заявка № ',
       'Дата бронирования', 'Дата начала тура', 'Год', 'Ночей', 'Вид тура',
       'Тип заявки', 'Страна тура', 'Регион отеля', 'Туристов',
       'Туристы, взрослые', 'Туристы, дети', 'Младенцы', 'Звездность',
       'Состав группы (Заявка) (Заявка)', 'Сумма в $', 'Наименование тура',
       'ИД клиента', 'Глубина продаж', 'Тип питания', 'Город Отеля',
       'Город отправления'],
      dtype='object')

# 1. Обогащение данных

In [5]:
big_mac_index = pd.read_csv("data/big_mac_index.csv", header=None)
big_mac_index.columns = ["Страна", "Индекс Биг-Мака"]
big_mac_index.head(3)

,Страна,Индекс Биг-Мака
0,Турция,145.0
1,Россия,135.0
2,ОАЭ,286.0


In [6]:
salary_by_city = pd.read_csv("data/zp_by_city.csv", index_col=0)
salary_by_city.columns = ["departure_city", "Средняя зарплата"]
salary_by_city.head(3)

,departure_city,Средняя зарплата
0,Красноярск,62846
1,Тюмень,93716
2,Калининград,43650


In [7]:
df = pd.merge(df, big_mac_index, left_on="Страна тура", right_on="Страна", how="outer")
df = pd.merge(
    df,
    salary_by_city,
    left_on="Город отправления",
    right_on="departure_city",
    how="outer",
)

# 1. Предобработка

In [8]:
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm

features_categorical = [
    "Пол",
    "Вид тура",
    "Тип заявки",
    "Страна тура",
    "Регион отеля",
    "Состав группы (Заявка) (Заявка)",
    "Тип питания",
    "Город отправления",
    "Город Отеля",
]

features_real = [
    "Возраст клиента",
    "Ночей",
    "Туристов",
    "Туристы, взрослые",
    "Туристы, дети",
    "Младенцы",
    "Сумма в $",
    "Глубина продаж",
    "Звездность",
    "Индекс Биг-Мака",
    "Средняя зарплата",
]

In [9]:
df_prep = df.copy()

In [10]:
features_categorical_mappers = {}


for col in tqdm(features_categorical):
    label_encoder = LabelEncoder()
    label_encoder.fit(df[col])
    features_categorical_mappers[col] = label_encoder
    df_prep[col] = label_encoder.transform(df[col])

  0%|          | 0/9 [00:00<?, ?it/s]

In [11]:
stars_mapping = {
    "4*": 4,
    "5*": 5,
    "3*": 3,
    "3*-4*": 3.5,
    "HV1": 5,
    "2*": 2,
    "0": 0,
    "3*+": 3.5,
    "-": None,
    "apt": 3,
    "nan": None,
    "HV": 5,
    "4*+": 4.5,
    "Без звёзд": 0,
    "1*": 1,
    "2*super": 3,
    "2*+": 2,
    "HV2": 4,
    "5* Deluxe": 5,
    "Std Apt": 3,
    "BOUTIQUE": 5,
    "Guest House": 3,
    "5*+": 5,
    "2*-3*": 2.5,
    "3*super": 4,
    "S-CLASS": 3.5,
    "cat B": 3,
    "SC": 3.5,
    "4*-5*": 4.5,
    "cat A": 4,
}
df_prep["Звездность"] = df_prep["Звездность"].map(stars_mapping)

In [12]:
df_prep[features_real] = df_prep[features_real].fillna(df_prep[features_real].mean())

# 3. train/test split

In [13]:
df_prep["Дата бронирования"] = pd.to_datetime(df_prep["Дата бронирования"])

df_train = df_prep.sort_values("Дата бронирования").drop_duplicates("ИД клиента")
df_test = df_prep[~df_prep.index.isin(df_train.index)]

all_features = list(set(features_categorical + features_real))

df_train = df_train[all_features]
df_test = df_test[all_features]
df_full = df_prep[all_features]

df_train.shape, df_test.shape, df_full.shape

((193276, 20), (16333, 20), (209609, 20))